# Part 1: Data cleaning and processing

Data preprocessing done in bash:
    * cut -d',' -f4- Sentiment\ Analysis\ Dataset.csv > justTweet.csv   
    * cut -d',' -f2 Sentiment\ Analysis\ Dataset.csv > justSentiment.csv 
    * paste -d',' justSentiment.csv justTweet.csv > sentimentTweet.csv
Data too large to put on git, you'll have to do the above preprocessing steps yourself to use the data for the following script.

In [81]:
f = open('./sentimentTweet.csv', 'r')
labels = []
tweets = []

for line in f:
    labels.append(line[0])
    tweets.append(line[2:])

In [82]:
corpus = {}

for i in range(len(labels)):
    corpus.update({i:(labels[i], tweets[i])})

In [149]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

import sys
reload(sys)
sys.setdefaultencoding('utf-8')


stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
#stemmer = PorterStemmer() 

''' I am not using the stemmer right now because the lemmatizer seems to be more appropriate for the task.
Feel free to switch it around. '''

def clean(doc):
    #doc = " ".join(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    #singles = " ".join(stemmer.stem(plural) for plural in punc_free.split())
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [150]:
new_corpus = {}
#doc_clean = [clean(doc).split() for doc in doc_complete]   
for i in range(1, len(corpus)):
    doc = (corpus[i])[1]
    doc = clean(doc).split()
    new_corpus.update({i-1:(labels[i], doc)})

In [153]:
new_corpus[0], (new_corpus[0])[1]

(('0', [u'sad', u'apl', u'friend']), [u'sad', u'apl', u'friend'])

# Part 2: Tf-Idf

In [151]:
tweet_data = []
for i in range(len(new_corpus)):
    tweet_data.append((new_corpus[i])[1])

In [152]:
tweet_data[0]

[u'sad', u'apl', u'friend']

In [143]:
from gensim import corpora, models, similarities

# Creating the term dictionary of the corpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(tweet_data)
dictionary.save('/tmp/tweets.dict') 

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(tweet) for tweet in tweet_data]
corpora.MmCorpus.serialize('/tmp/tweets.mm', doc_term_matrix)

In [145]:
# Initialize TF-IDF model and transform Text with TF-IDF
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

In [146]:
# Create similarity matrix of all documents/tweets
index = similarities.MatrixSimilarity(tfidf[doc_term_matrix])
index.save('/tmp/deerwester.index')

In [147]:
sims = index[corpus_tfidf]

In [148]:
sims

array([[ 1.00000012,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.99999994,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.99999994,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.99999994,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]], dtype=float32)

In [139]:
''' 
For tutorial on using LDA model with Gensim:
https://radimrehurek.com/gensim/wiki.html#latent-dirichlet-allocation 
'''